<a href="https://colab.research.google.com/github/ThiagSampaio/DeepLearning/blob/main/Regress%C3%A3o_Log%C3%ADstica_com_Rede_Neural.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Regressão Logística com Rede Neural

Vamos construir um classificador de regressão logística para reconhecer gatos.

** O que será mostrado: **
- Construir uma arquitetura geral de um algoritmo de aprendizado, incluindo:
    - Inicializando os parâmetros
    - Calculando a função custo e seu gradiente
    - Usando um algoritmo de otimização(gradiente descente)
- Pegar as três funções acima e aplicá-las em uma função modelo principal, na ordem correta.

## Tabela de Conteúdos
- [1 - Pacotes](#1)
- [2 - Olhando o problema](#2)
    - [Demonstração 1](#ex-1)
    - [Demonstração 2](#ex-2)
- [3 - Arquitetura geral de um algoritmo de aprendizado](#3)
- [4 - Construindo as partes do nosso algoritmo](#4)
    - [4.1 - Funções úteis](#4-1)
        - [Demonstração 3 - sigmoid](#ex-3)
    - [4.2 - Inicializando parâmetros](#4-2)
        - [Demonstração 4 - inicializando_com_zeros](#ex-4)
    - [4.3 - Propagação direta e inversa](#4-3)
        - [Demonstração 5 - propagação](#ex-5)
    - [4.4 - Optimization](#4-4)
        - [Demonstração 6 - otimização](#ex-6)
        - [Demonstração 7 - predição](#ex-7)
- [5 - Aglutinar todas as funções em um modelo](#5)
    - [Demonstração 8 - modelo](#ex-8)
- [6 - Análises](#6)
- [7 - Testando com sua prória imagem](#7)

